In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("M2M3逾期明细.xlsx",dtype={'贷款编号':'O','SA工号':'O'})
data2 = pd.read_excel("M2+逾期明细.xlsx",dtype={'贷款编号':'O','SA工号':'O'})

In [3]:
m2 = data[data["首次M2"]==1]

In [4]:
m2_over_count = pd.pivot_table(m2,index=['SA工号'],values=['贷款编号'],aggfunc=[len])

In [5]:
def yikou():
    # 汇总首次M2和M2+的已经扣罚和退还的
    data11 = pd.read_excel("../扣罚汇总/M2+扣罚汇总.xlsx",dtype={'贷款编号':'O','SA工号':'O'})
    data22 = pd.read_excel("../扣罚汇总/首次M2扣罚汇总.xlsx",dtype={'贷款编号':'O','SA工号':'O'})
    yikou = pd.concat([data11, data22])
    return yikou
yikou = yikou()

#M2已经扣的剔除
yikou = yikou[['贷款编号','扣押月份','退还月份']]
data2 = pd.merge(data2,yikou,on="贷款编号",how="left")


In [6]:
len(data2)

25180

In [7]:
for i in range(len(data2)):
    if np.isnan(data2.loc[i,"扣押月份"]) == False and np.isnan(data2.loc[i,"退还月份"])==True:
        data2.drop([i],inplace=True)
    else:
        pass
        
data2 = data2.reset_index(drop=True)  # 对索引重置

In [8]:
m2_zc = pd.read_excel("../首次m2/m2首次注册数.xlsx",dtype={'SA工号':'O'})

In [9]:
m2_over_rate = pd.merge(m2_zc,m2_over_count,on="SA工号",how="left")

d:\Anaconda3\envs\lihuaqi\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
for i in range(len(m2_over_rate)):
    m2_over_rate.loc[i,'首次M2逾期率'] = (m2_over_rate.iloc[i,3] /m2_over_rate.iloc[i,2])*100

In [11]:
m2_over_rate = m2_over_rate[['SA工号','首次M2逾期率']]

In [12]:
data_first_m2 = m2.copy()
data_first_m2 = data_first_m2[['SA工号','SA姓名']]
data_m2_plus = data2.copy()
data_m2_plus = data_m2_plus[['SA工号','SA姓名']]

data_all = pd.concat([data_first_m2,data_m2_plus]).drop_duplicates(['SA工号'])
data_all = data_all.reset_index(drop=True)

In [13]:

overdue_rate_m3 = pd.read_excel("../首次M2/Overdue_rate_M3.xlsx",dtype={'SA工号':'O'})
overdue_rate_m3 = overdue_rate_m3[['SA工号','M3+逾期率(%)']]
people_list = pd.read_excel("../首次M2/people_listing.xlsx",dtype={'SA工号':'O','入岗日期转换':np.datetime64})
people_list = people_list[['SA工号','姓名','在岗状态','入岗日期转换','结算日期','在职天数']]
data_all = pd.merge(data_all,people_list, on="SA工号",how="left",suffixes=('','_y'))
data_all = pd.merge(data_all,m2_over_rate,on="SA工号",how="left")
data_all = pd.merge(data_all,overdue_rate_m3,on="SA工号",how="left")

In [14]:
data_all

,SA工号,SA姓名,姓名,在岗状态,入岗日期转换,结算日期,在职天数,首次M2逾期率,M3+逾期率(%)
0,23100241,文军,文军,在职,2017-08-09,2018-08-31,388,9.090909,8.75
1,04110014,邓康,邓康,在职,2018-04-04,2018-08-31,150,10.344828,1.75
2,19170028,苏庆飞,苏庆飞,在职,2016-11-25,2018-08-31,645,15.789474,7.95
3,19080078,孙海波,孙海波,离职,2018-05-11,2018-08-31,113,11.111111,12.50
4,26260186,詹振华,詹振华,离职,2018-04-10,2018-08-31,144,57.142857,13.33
5,23100286,王杰,王杰,在职,2018-04-08,2018-08-31,146,9.259259,5.06
6,12250777,李颖,李颖,在职,2018-04-25,2018-08-31,129,8.695652,0.00
7,12170217,刘倩,刘倩,离职,2018-01-30,2018-08-31,214,18.181818,11.34
8,08260103,黄子真,黄子真,在职,2018-02-13,2018-08-31,200,11.111111,0.00
9,25080068,余欢欢,余欢欢,在职,2015-09-08,2018-08-31,1089,3.846154,4.08


In [15]:
for i in range(len(data_all)):
    if np.isnan(data_all.loc[i,"首次M2逾期率"]):
        data_all.loc[i,"首次M2逾期率"] = 0
    else:
        pass

for i in range(len(data_all)):
    if np.isnan(data_all.loc[i,"M3+逾期率(%)"]):
        data_all.loc[i,"M3+逾期率(%)"] = 0
    else:
        pass

In [16]:
data_all

,SA工号,SA姓名,姓名,在岗状态,入岗日期转换,结算日期,在职天数,首次M2逾期率,M3+逾期率(%)
0,23100241,文军,文军,在职,2017-08-09,2018-08-31,388,9.090909,8.75
1,04110014,邓康,邓康,在职,2018-04-04,2018-08-31,150,10.344828,1.75
2,19170028,苏庆飞,苏庆飞,在职,2016-11-25,2018-08-31,645,15.789474,7.95
3,19080078,孙海波,孙海波,离职,2018-05-11,2018-08-31,113,11.111111,12.50
4,26260186,詹振华,詹振华,离职,2018-04-10,2018-08-31,144,57.142857,13.33
5,23100286,王杰,王杰,在职,2018-04-08,2018-08-31,146,9.259259,5.06
6,12250777,李颖,李颖,在职,2018-04-25,2018-08-31,129,8.695652,0.00
7,12170217,刘倩,刘倩,离职,2018-01-30,2018-08-31,214,18.181818,11.34
8,08260103,黄子真,黄子真,在职,2018-02-13,2018-08-31,200,11.111111,0.00
9,25080068,余欢欢,余欢欢,在职,2015-09-08,2018-08-31,1089,3.846154,4.08


In [17]:
for i in range(len(data_all)):
    if data_all.loc[i,"在职天数"] >= 90 :
        if data_all.loc[i,"首次M2逾期率"] < 5.0:
            if data_all.loc[i,"M3+逾期率(%)"] < 7.0:
                data_all.loc[i,"是否免除扣罚"] = 1
            else:
                data_all.loc[i,"是否免除扣罚"] = 0
        else:
             data_all.loc[i,"是否免除扣罚"] = 0
    else:
        if data_all.loc[i,"首次M2逾期率"] < 4.0:
            data_all.loc[i,'是否免除扣罚'] = 1
        else:
            data_all.loc[i,"是否免除扣罚"] = 0
data_all = data_all[["SA工号","SA姓名","在岗状态","入岗日期转换","结算日期","在职天数","M3+逾期率(%)","首次M2逾期率","是否免除扣罚"]]
data_all.to_excel('../数据输出/SA扣罚标准.xlsx')
